In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import Tokenizer
import joblib





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/mysampledata/emails.csv')
df

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset into a pandas DataFrame
# Assuming your DataFrame is named 'df'
# Make sure the first column contains the subject of the email and the second column contains the label (0 for spam, 1 for not spam)

# Split the data into features and labels
X = df.iloc[:, 0]  # Subject of the email
y = df.iloc[:, 1]  # Spam or not spam label

# Convert labels to numeric form
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)

joblib.dump(tokenizer, 'tokenizer.joblib')


X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
max_length = 100  # Choose a suitable length based on your dataset
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(max_length,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, epochs=10, batch_size=64, validation_data=(X_test_pad, y_test))


Epoch 1/10
72/72 [==============================] - 1s 4ms/step - loss: 335.2303 - accuracy: 0.6032 - val_loss: 162.4245 - val_accuracy: 0.6195
Epoch 2/10
72/72 [==============================] - 0s 2ms/step - loss: 124.6511 - accuracy: 0.6787 - val_loss: 114.4727 - val_accuracy: 0.6920
Epoch 3/10
72/72 [==============================] - 0s 2ms/step - loss: 76.6024 - accuracy: 0.7119 - val_loss: 89.2448 - val_accuracy: 0.6824
Epoch 4/10
72/72 [==============================] - 0s 2ms/step - loss: 48.4731 - accuracy: 0.7420 - val_loss: 80.7424 - val_accuracy: 0.6483
Epoch 5/10
72/72 [==============================] - 0s 2ms/step - loss: 33.8189 - accuracy: 0.7722 - val_loss: 72.0401 - val_accuracy: 0.6832
Epoch 6/10
72/72 [==============================] - 0s 2ms/step - loss: 25.9543 - accuracy: 0.7879 - val_loss: 69.0902 - val_accuracy: 0.6283
Epoch 7/10
72/72 [==============================] - 0s 2ms/step - loss: 18.4985 - accuracy: 0.8186 - val_loss: 62.0350 - val_accuracy: 0.6763
Ep

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred_probs = model.predict(X_test_pad)

y_pred = (y_pred_probs > 0.5).astype("int32")

y_pred = y_pred.ravel()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

conf_matrix = confusion_matrix(y_test, y_pred)

print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("\nConfusion Matrix:")
print(conf_matrix)


36/36 [==============================] - 0s 929us/step
Evaluation Metrics:
Accuracy: 0.675392670157068
Precision: 0.3835227272727273
Recall: 0.46551724137931033
F1-score: 0.4205607476635514

Confusion Matrix:
[[639 217]
 [155 135]]


In [ ]:
import joblib

# Save the trained model
joblib.dump(model, 'email_spam_classifier_model.joblib')

['email_spam_classifier_model.joblib']

In [ ]:
model.save('email_spam_classifier_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Apply the patch to allow nested use of asyncio.run
nest_asyncio.apply()

app = FastAPI()

# Load the saved model
model = tf.keras.models.load_model('email_spam_classifier_model.h5')

# Load the tokenizer
tokenizer = joblib.load('tokenizer.joblib')

class Email(BaseModel):
    subject: str

@app.post("/predict/")
def predict(email: Email):
    # Preprocess the email subject
    seq = tokenizer.texts_to_sequences([email.subject])
    padded = pad_sequences(seq, maxlen=100, padding='post')

    # Make prediction
    pred = model.predict(padded)
    is_spam = int(pred[0][0] > 0.5)  # Convert numpy.int64 to Python int

    # Return the result
    return {"subject": email.subject, "is_spam": is_spam}

# Set up ngrok with your authtoken
ngrok.set_auth_token('2faAS3AIxw8OoeA3p6b6fYeelS5_3VqBdRUxXLLNpC56HDEuC')

# Start an ngrok tunnel to the uvicorn server
tunnel = ngrok.connect(8001)
print('Public URL:', tunnel.public_url)

# Use uvicorn to run the app, specifying host and port
uvicorn.run(app, host="127.0.0.1", port=8001)
